In [1]:
import czapi.api as api
import pandas as pd
from time import sleep
import ipywidgets

In [2]:
columns = ['team_name','href','hammer_start','score','final_score','hammer_progression','relative_score','guid']
def make_frame(event:api.Event)->pd.DataFrame:
    return pd.concat([pd.DataFrame(data=boxscore,columns=columns) for boxscore in event.get_flat_boxscores()],axis=0,sort=True).reset_index(drop=True)


In [3]:
def explode_frame(df:pd.DataFrame)->pd.DataFrame:
    df['end'] = df.hammer_progression.apply(lambda x: range(1,len(x)+1))
    return df.explode(column=['hammer_progression','relative_score','end']).reset_index(drop=True)

In [4]:
def transform_frame(df:pd.DataFrame)->pd.DataFrame:
    return df.assign(

     final_score = df.final_score.astype(int)
    ,hammer_progression = df.hammer_progression.astype(bool)
    ,hammer_start = df.hammer_start.astype(bool)
    ,relative_score = df.relative_score.astype(int)
    ,ends = df.end.astype(int)
    ,team_id = df.href.str.extract(pat='^.*teamid=(\d*)[#&].*').astype(int)

)


In [5]:
def get_games_played(df:pd.DataFrame)->pd.DataFrame:
    return df.groupby(['team_name','team_id'])['guid'].nunique().sort_values(ascending=False).to_frame().rename({'guid':'games_played'},axis=1)


In [6]:
def get_hammer_to_start_game(df:pd.DataFrame)->pd.DataFrame:

    return df[['team_name','team_id','guid','hammer_start']].drop_duplicates().groupby(['team_name','team_id'])['hammer_start'].mean().sort_values(ascending=False).to_frame().rename({'hammer_start':'pct_start_with_hammer'},axis=1)



In [7]:
def get_points_scored(df:pd.DataFrame)->pd.DataFrame:

    return df[['team_name','team_id','guid','final_score']].drop_duplicates().groupby(['team_name','team_id'])['final_score'].mean().sort_values(ascending=False).to_frame().rename({'final_score':'pts_for'},axis=1)


In [8]:
print('Input CurlingZone event ID here:')
event_id = ipywidgets.widgets.BoundedIntText(min=1,max=10000)
event_id

Input CurlingZone event ID here:


BoundedIntText(value=1, max=10000, min=1)

In [13]:
run = ipywidgets.widgets.Button(description="Run report!")
out = ipywidgets.widgets.Output()

def run_report(change)->api.Event:
    
    out.clear_output()
    event = api.Event(cz_event_id = event_id.value,verbose=True,delay=3)
    df = make_frame(event=event)
    df = explode_frame(df=df)
    df = transform_frame(df=df)
    

    
    # games played
    games_played = get_games_played(df=df)
    out.append_display_data(games_played)
    
    # hammer to start game
    hammer_to_start_game = get_hammer_to_start_game(df=df)
    out.append_display_data(hammer_to_start_game)
    
    # points score
    
    points_scored = get_points_scored(df=df)
    out.append_display_data(points_scored)
    
#     print(df.head())
#     print(df.dtypes)

run.on_click(run_report)

In [14]:
ipywidgets.widgets.VBox([run,out])

Scraping draw 1.
Scraping draw 2.
Scraping draw 3.
Scraping draw 4.
Scraping draw 5.
Scraping draw 6.
Scraping draw 7.
Scraping draw 8.
